In [1]:
import xarray as xr
import s3fs
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
from datetime import datetime
import numpy as np
from dask.distributed import Client, LocalCluster

In [2]:
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))

In [3]:
def paths_to_datetimeindex(paths):
    return  [datetime.strptime(date.split('/')[-1].split('-')[3], '%Y%m%d') for date in paths]

In [5]:
def read_daily_files(year):
    for month in range (1,13):
        remote_files = 's3://escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-%i%s*.nc'%(year,str(month).zfill(2))
        remote_files = s3.glob(remote_files)
        fileset = [s3.open(file) for file in remote_files]
        time_var = xr.Variable('time', paths_to_datetimeindex(remote_files))
        dset =xr.open_mfdataset(fileset, concat_dim="time", combine = "nested")
        dset = dset.assign_coords(time=time_var)
        dset.to_netcdf('AMSR2_ASI_sea_ice_concentration_%i%s.nc'%(year,str(month).zfill(2)))
        del remote_files, dset, time_var

In [6]:
[read_daily_files(year) for year in range(2012,2020)]


KeyboardInterrupt

